Recommending movies to a user based on multiple movie ratings by that user.

In [1]:
import pandas as pd
from scipy import sparse

In [2]:
ratings = pd.read_csv('ratings.csv')

movies = pd.read_csv('movies.csv')

ratings = pd.merge(movies,ratings).drop(['genres','timestamp'],axis=1)

print(ratings.shape)
ratings.head()

(20000263, 4)


,movieId,title,userId,rating
0,1,Toy Story (1995),3,4.0
1,1,Toy Story (1995),6,5.0
2,1,Toy Story (1995),8,4.0
3,1,Toy Story (1995),10,4.0
4,1,Toy Story (1995),11,4.5


In [4]:
ratings_short = ratings.groupby('userId').filter(lambda x:len(x)>500)

In [5]:
userRatings = ratings_short.pivot_table(index=['userId'],columns=['title'],values='rating')

print("Before: ",userRatings.shape)

Before:  (7441, 26034)


In [6]:
userRatings = userRatings.dropna(thresh=500, axis=1).fillna(0,axis=1)
#userRatings.fillna(0, inplace=True)

print("After: ",userRatings.shape)

After:  (7441, 3129)


In [7]:
corrMatrix = userRatings.corr(method='pearson')

In [9]:
corrMatrix

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zack and Miri Make a Porno (2008),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",1.000000,-0.058743,0.195622,0.049851,0.117242,0.007373,0.085219,0.128579,0.043407,0.001783,...,0.015103,-0.004966,-0.030372,0.091073,-0.021621,-0.024426,0.056357,0.037756,0.013986,0.312113
(500) Days of Summer (2009),-0.058743,1.000000,0.029838,-0.023804,0.098423,0.157227,0.026645,-0.041274,0.024447,0.045312,...,0.365706,0.073790,0.320763,-0.041645,0.377384,0.471626,0.144213,0.025234,0.051881,-0.108414
*batteries not included (1987),0.195622,0.029838,1.000000,0.015554,0.088953,0.068397,0.086743,0.130191,0.073934,0.000517,...,0.049507,0.047609,0.008399,0.018694,0.017090,0.079328,0.108357,0.055524,0.098577,0.122783
...And Justice for All (1979),0.049851,-0.023804,0.015554,1.000000,-0.049461,-0.027421,0.008002,0.045367,0.023374,0.128570,...,-0.017157,0.104201,0.034852,0.062717,0.008764,-0.046200,-0.053464,0.041050,-0.030079,0.058935
10 Things I Hate About You (1999),0.117242,0.098423,0.088953,-0.049461,1.000000,0.058934,0.186032,0.132902,0.125756,-0.054386,...,0.103428,-0.072345,-0.000006,0.035006,0.016977,0.052582,0.203284,0.014604,0.155816,0.079113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zombieland (2009),-0.024426,0.471626,0.079328,-0.046200,0.052582,0.251491,0.032982,-0.036572,0.021510,0.012646,...,0.413400,0.024469,0.283946,-0.063027,0.356198,1.000000,0.202927,0.045705,0.141669,-0.069651
Zoolander (2001),0.056357,0.144213,0.108357,-0.053464,0.203284,0.081659,0.088025,0.029351,0.065523,-0.005101,...,0.201272,0.053759,0.068373,0.018389,0.168990,0.202927,1.000000,0.068039,0.220427,0.106849
eXistenZ (1999),0.037756,0.025234,0.055524,0.041050,0.014604,0.009548,-0.045704,-0.050534,-0.017715,0.008366,...,0.042291,0.070797,0.023579,0.143652,0.079822,0.045705,0.068039,1.000000,0.086525,-0.013687


In [8]:
def get_similar(movie_name,rating):
    similar_ratings = corrMatrix[movie_name]*(rating-2.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_ratings

In [10]:
romantic_lover = [("(500) Days of Summer (2009)",5),("Alice in Wonderland (2010)",3),("Aliens (1986)",1),
                  ("2001: A Space Odyssey (1968)",2)]

similar_movies = pd.DataFrame()

for movie,rating in romantic_lover:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)

,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zack and Miri Make a Porno (2008),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
0,-0.146857,2.500000,0.074596,-0.059511,0.246058,0.393069,0.066612,-0.103186,0.061118,0.113281,...,0.914265,0.184476,0.801908,-0.104112,0.943461,1.179066,0.360533,0.063084,0.129703,-0.271035
1,-0.012206,0.183628,0.030571,-0.016349,0.033318,0.114141,0.049960,0.015579,0.033264,0.007960,...,0.143846,0.006606,0.105152,-0.034317,0.127814,0.202669,0.054321,0.011218,0.046226,-0.038416
2,-0.123488,0.114932,-0.089831,-0.069127,0.147441,-0.022653,0.095664,-0.005657,0.075632,-0.055183,...,0.046197,0.008976,-0.032297,-0.114850,-0.028294,-0.051802,0.012076,-0.266868,-0.081263,-0.179669
3,-0.002841,0.024039,-0.017036,-0.038074,0.081986,0.019170,0.038295,-0.014870,0.029994,-0.092319,...,0.023113,-0.079637,-0.016777,-0.049627,-0.030010,0.014960,0.020342,-0.079122,0.035067,-0.022130


In [11]:
similar_movies.sum().sort_values(ascending=False).head(20)

(500) Days of Summer (2009)                    2.822599
Up in the Air (2009)                           1.669439
Juno (2007)                                    1.609255
Social Network, The (2010)                     1.609095
Hangover, The (2009)                           1.597525
Inception (2010)                               1.585177
Slumdog Millionaire (2008)                     1.543975
Up (2009)                                      1.525432
Black Swan (2010)                              1.519998
Inglourious Basterds (2009)                    1.508068
Alice in Wonderland (2010)                     1.496876
Shutter Island (2010)                          1.475853
50/50 (2011)                                   1.472370
Avatar (2009)                                  1.434932
Scott Pilgrim vs. the World (2010)             1.427207
Crazy, Stupid, Love. (2011)                    1.423658
Sherlock Holmes (2009)                         1.422359
Easy A (2010)                                  1

In [12]:
action_lover = [("Amazing Spider-Man, The (2012)",5),("Mission: Impossible III (2006)",4),("Toy Story 3 (2010)",2),
                ("2 Fast 2 Furious (Fast and the Furious 2, The) (2003)",4)]

similar_movies = pd.DataFrame()

for movie,rating in action_lover:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)
similar_movies.sum().sort_values(ascending=False).head(20)

Amazing Spider-Man, The (2012)                           2.897146
Mission: Impossible III (2006)                           2.447813
2 Fast 2 Furious (Fast and the Furious 2, The) (2003)    2.218221
Avengers, The (2012)                                     2.035513
Thor (2011)                                              2.027263
Mission: Impossible - Ghost Protocol (2011)              1.985969
Captain America: The First Avenger (2011)                1.980741
X-Men: First Class (2011)                                1.967445
Iron Man 2 (2010)                                        1.926796
Iron Man 3 (2013)                                        1.921735
Dark Knight Rises, The (2012)                            1.892805
Transformers (2007)                                      1.815593
X-Men Origins: Wolverine (2009)                          1.810056
Captain America: The Winter Soldier (2014)               1.801604
Man of Steel (2013)                                      1.780152
X-Men: The